In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LSTM Experiment

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.data import Batch

In [47]:
train_npz = np.load('../../train.npz')
train_data = train_npz['data']
test_npz  = np.load('../../test_input.npz')
test_data  = test_npz['data']

In [48]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [49]:
import sys
sys.path.append('..')
from TrajectoryDataset import TrajectoryDatasetTrain, TrajectoryDatasetTest
from utils import train_model

sys.path.append('../..')
from models.lstm import LSTMModel

In [50]:
torch.manual_seed(251)
np.random.seed(42)

scale = 5.0

N = len(train_data)
val_size = int(0.05 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


In [ ]:
model = LSTMModel()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25) # You can try different schedulers
early_stopping_patience = 10
criterion = nn.MSELoss()

In [ ]:
model = model.to(device)
train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    device, 
    optimizer, 
    criterion, 
    scheduler, 
    early_stopping_patience,
    "lstm_weights"
)

Epoch:   1%|          | 1/100 [00:06<09:55,  6.01s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE   1.7489 | val normalized MSE   0.5056, | val MAE   2.7305 | val MSE  24.7755


Epoch:   2%|▏         | 2/100 [00:11<09:00,  5.52s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE   0.4617 | val normalized MSE   0.3849, | val MAE   2.2651 | val MSE  18.8610


Epoch:   3%|▎         | 3/100 [00:16<08:37,  5.33s/epoch]

...

Epoch 057 | Learning rate 0.000063 | train normalized MSE   0.1844 | val normalized MSE   0.1882, | val MAE   1.4218 | val MSE   9.2214


Epoch:  59%|█████▉    | 59/100 [05:00<03:19,  4.88s/epoch]

Epoch 058 | Learning rate 0.000063 | train normalized MSE   0.1848 | val normalized MSE   0.1884, | val MAE   1.4090 | val MSE   9.2333


Epoch:  59%|█████▉    | 59/100 [05:05<03:32,  5.18s/epoch]

Epoch 059 | Learning rate 0.000016 | train normalized MSE   0.1839 | val normalized MSE   0.1885, | val MAE   1.4240 | val MSE   9.2383
Early stop!


In [ ]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

lstm_weights = torch.load("lstm_weights.pt")
model.load_state_dict(lstm_weights)

# also save entire model
torch.save(model, "full_lstm_model.pt")

test_dataset.generate_submission_predictions(model, device, output_file_name='lstm_submission')